In [37]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [38]:
%pip install google-cloud-bigquery
%pip install pandas
%pip install db-dtypes
%pip install pandas-gbq

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [39]:
from google.cloud import bigquery
import pandas as pd

from utils.import_table import import_table
from utils.export import export_table


In [40]:
# Initialiser le client BigQuery
client = bigquery.Client.from_service_account_json('../../key.json')

In [41]:
date = import_table(client, 'date', dataset_id='warehouse_test')
display(date.head())

/Users/antoine/miniconda3/envs/AI/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,id,date
0,1,1970-01-08
1,2,1970-01-22
2,3,1970-02-15
3,4,1970-02-24
4,5,1970-03-06


In [42]:
sms = import_table(client, 'StockMovementStatus').sort_values(by='id')

# replace date in sms by date_id of date 
sms = sms.merge(date, on='date', how='left')
sms = sms.drop(columns=['date'])
sms = sms.rename(columns={'id_y': 'date_id', 'id_x': 'id'})
display(sms.head())


/Users/antoine/miniconda3/envs/AI/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,id,stock_movement_id,status,date_id
0,1,1,pending,956
1,2,2,pending,732
2,3,2,processing,734
3,4,2,shipped,736
4,5,2,delivered,737


In [43]:
# create col pending_date, processing_date, shipped_date, delivered_date, cancelled_date with date_id in sms
pending = sms[sms['status'] == 'pending']
processing = sms[sms['status'] == 'processing']
shipped = sms[sms['status'] == 'shipped']
delivered = sms[sms['status'] == 'delivered']
cancelled = sms[sms['status'] == 'cancelled']

In [49]:
sm = import_table(client, 'StockMovement') 
sm = sm.drop(columns=['date'])

/Users/antoine/miniconda3/envs/AI/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [50]:
def merge_date(sm, df, label):
    sm = sm.merge(df[['stock_movement_id', 'date_id']], left_on='id', right_on='stock_movement_id', how='left')
    sm = sm.rename(columns={'date_id': label})
    sm = sm.drop(columns=['stock_movement_id'])
    return sm

In [51]:
sm = merge_date(sm, pending, 'pending_date_id')
sm = merge_date(sm, processing, 'processing_date_id')
sm = merge_date(sm, shipped, 'shipped_date_id')
sm = merge_date(sm, delivered, 'delivered_date_id')
sm = merge_date(sm, cancelled, 'cancelled_date_id')

display(sm.head())

,id,product_id,from_location_id,to_location_id,order_id,supervised_by,quantity,pending_date_id,processing_date_id,shipped_date_id,delivered_date_id,cancelled_date_id
0,2,150,24,56,2,1,37,732,734,736,737,<NA>
1,5,28,91,99,5,1,87,20,21,22,23,<NA>
2,6,44,38,63,6,1,3,136,<NA>,<NA>,<NA>,<NA>
3,26,178,60,38,26,1,62,48,50,51,<NA>,<NA>
4,48,191,66,86,48,1,20,559,560,<NA>,<NA>,<NA>


In [52]:
sm['is_cancelled'] = sm['cancelled_date_id'].notnull()
display(sm.head())

,id,product_id,from_location_id,to_location_id,order_id,supervised_by,quantity,pending_date_id,processing_date_id,shipped_date_id,delivered_date_id,cancelled_date_id,is_cancelled
0,2,150,24,56,2,1,37,732,734,736,737,<NA>,False
1,5,28,91,99,5,1,87,20,21,22,23,<NA>,False
2,6,44,38,63,6,1,3,136,<NA>,<NA>,<NA>,<NA>,False
3,26,178,60,38,26,1,62,48,50,51,<NA>,<NA>,False
4,48,191,66,86,48,1,20,559,560,<NA>,<NA>,<NA>,False


In [53]:
# Variables pour BigQuery
dataset_name = "warehouse_test" # Nom du dataset à créer/exporter
table_name = "f_stock_movement" # Nom de la table à créer


In [54]:
export_table(client, sm, dataset_name, table_name)

Dataset warehouse_test déjà existant.


100%|██████████| 1/1 [00:00<00:00, 9532.51it/s]
